# Denison DA210/CS181 Homework 4.a - Step 2

Before you turn this notebook in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

#### Set credentials

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json"):
    """
    Using directory and filename parameters, open a credentials file
    and obtain the two parts needed for a connection string to
    a local provider using the "sqlite" dictionary within
    an outer dictionary.  
        
    Returns: a scheme and a dbfile.
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D["sqlite"]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds()
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish connection from client to server

In [4]:
%sql $cstring

---

## Part D: Single-Table Column Projection

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises._

**Q3:** Using the table `countries`, give the SQL query you would use to obtain a set of all country names. There should be no other columns projected.

In [9]:
query3 = """
SELECT country
FROM countries
"""


resultset3 = %sql $query3
resultdf3 = resultset3.DataFrame()
resultdf3.head()

 * sqlite:///../../dbfiles/book.db
Done.


,country
0,Aruba
1,Afghanistan
2,Angola
3,Albania
4,Andorra


In [10]:
# Testing cell
assert len(resultdf3) == 217
assert 'Aruba' in list(resultdf3['country'])

**Q4:** Project the year, code, population, and number of cell-phones from `indicators`.  Rename each column name to have a capital first letter.

In [12]:
query4 = """
SELECT year AS Year, code AS Code, pop AS Pop, cell AS Cell
FROM indicators
"""


resultset4 = %sql $query4
resultdf4 = resultset4.DataFrame()
resultdf4.head()

 * sqlite:///../../dbfiles/book.db
Done.


,Year,Code,Pop,Cell
0,1960,ABW,0.05,0.0
1,1960,AFG,9.00,0.0
2,1960,AGO,5.45,0.0
3,1960,ALB,1.61,0.0
4,1960,AND,0.01,0.0


In [13]:
# Testing cell
assert len(resultdf4) == 12862
assert resultdf4.shape == (12862,4)
assert 'Pop' in resultdf4.columns
assert 'Cell' in resultdf4.columns

---

## Part E: Ordering Results

**Q5:** Using the SQL table `countries`, project all columns and produce a table of rows ordered by landmass, from smallest to largest.

In [23]:
query5 = """
SELECT * 
FROM countries 
ORDER BY land ASC 
"""


resultset5 = %sql $query5
resultdf5 = resultset5.DataFrame()
resultdf5.head()

 * sqlite:///../../dbfiles/book.db
Done.


,code,country,region,income,land
0,CUW,Curacao,Latin America & Caribbean,High income,NaN
1,MAF,St. Martin (French part),Latin America & Caribbean,High income,NaN
2,MCO,Monaco,Europe & Central Asia,High income,NaN
3,SDN,Sudan,Sub-Saharan Africa,Lower middle income,NaN
4,SSD,South Sudan,Sub-Saharan Africa,Low income,NaN


In [24]:
# Testing cell
assert len(resultdf5) == 217
assert resultdf5.iloc[0,0] == 'CUW'
assert resultdf5.iloc[-1,0] == 'RUS'

**Q6:** Use SQL to find the rows with the top 20 GDP values in `indicators`.  You should project all the columns.

In [25]:
query6 = """
SELECT * 
FROM indicators 
ORDER BY gdp DESC 
LIMIT 20
"""

resultset6 = %sql $query6
resultdf6 = resultset6.DataFrame()
resultdf6.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,2018,USA,327.17,20494.1,NaN,NaN,NaN,NaN
1,2017,USA,325.15,19485.4,78.5,391.60,2342670.0,1545610.0
2,2016,USA,323.07,18707.2,78.5,387.40,2189180.0,1453700.0
3,2015,USA,320.74,18219.3,78.7,382.31,2241660.0,1504570.0
4,2014,USA,318.39,17521.8,78.8,355.50,2347680.0,1620480.0


In [26]:
assert len(resultdf6) == 20
assert resultdf6.loc[0, 'year'] == 2018
assert resultdf6.loc[0, 'code'] == 'USA'
assert resultdf6.loc[0, 'gdp'] == 20494.1

**Q7:** Using a SQL query and the `indicators` table, answer the following question: what country and in what year was the greatest number of cell phones? Project `year`, `code`, and `cell`.

In [34]:
query7 = """
SELECT year, code, cell
FROM indicators 
ORDER BY cell DESC 
"""

resultset7 = %sql $query7
resultdf7 = resultset7.DataFrame()
resultdf7.head()


print("The greatest number of cell phones is " + str(resultdf7["cell"][0]) + " from " + str(resultdf7["code"][0]) + " in "+ str(resultdf7["year"][0]) +".")

 * sqlite:///../../dbfiles/book.db
Done.
The greatest number of cell phones is 1469.88 from CHN in 2017.


In [30]:
# Testing cell
assert resultdf7.loc[0,'cell'] == 1469.88
assert resultdf7.loc[0,'year'] == 2017
assert resultdf7.loc[0,'code'] == 'CHN'

**Q8:** Your answer to the previous question may have resulted in more than one record (row).  Project `year`, `code`, and `cell` for only the country with the greatest number of cell phones.  Your resulting DataFrame should have only one row.

In [31]:
query8 = """
SELECT year, code, cell
FROM indicators 
ORDER BY cell DESC 
LIMIT 1
"""

resultset8 = %sql $query8
resultdf8 = resultset8.DataFrame()
resultdf8.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,cell
0,2017,CHN,1469.88


In [35]:
# Testing cell
assert len(resultdf8) == 1
assert resultdf8.loc[0,'cell'] == 1469.88
assert resultdf8.loc[0,'year'] == 2017
assert resultdf8.loc[0,'code'] == 'CHN'

---

---

## Part F

**Q9:** How much time (in minutes/hours) did you spend on this homework assignment?

10 minutes

**Q10:** Who was your partner for this assignment?  If you worked alone, say so instead.

Alone